<a href="https://colab.research.google.com/github/jaimebaldeon/TSP-evolutionary-algorithm/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import statistics
import random

class TSP:
  def __init__(self, numVertices):
    #self.vertices = [n for n in range(1, numVertices+1)] #to accomplish natural values for vertices
    self.vertices = [n for n in range(numVertices)]
    matrix = np.random.randint(100, 500, size=(numVertices, numVertices))
    np.fill_diagonal(matrix, 0)
    self.distanceMatrix = matrix

class Individual:
  def __init__(self, tsp):
    self.order = np.random.permutation(tsp.vertices)
    self.α = 0.1

def fitness(tsp, ind):
  value = 0
  ii = 0
  for i in ind.order:
    # Sum up total distance for the individual's sequence of nodes 
    while (ii < ind.order.size - 1):
      pairVertices = ind.order[ii:ii+2]
      value += tsp.distanceMatrix[pairVertices[0], pairVertices[1]]
      ii += 1 
  # add the distance between first and last node
  value += tsp.distanceMatrix[ind.order[-1], ind.order[0]]
  return value

def optimize(tsp):
  λ = 100 # population size
  μ = 100 # offspring size
  iter =100

  # Initialization
  population = initialize(tsp,λ)
  offspring = []

  # Print initial Candidate solutions
  for ind in population:
    print("Candidate solution:  ", ind.order)

  # Print initial Population evaluation
  fitnesses = [fitness(tsp, ind) for ind in population]
  print("\nMean fitness:  ", statistics.mean(fitnesses))
  print("Best fitness:  ", min(fitnesses))

  for i in range(iter):
    # Recombination
    for ind in range(μ):
      # Selection
      parent1 = selection(tsp, population)
      parent2 = selection(tsp, population)
      offspring.append(recombination(parent1, parent2))
      mutate(offspring[ind])

    # Mutation
    for ind in population:
      mutate(ind)    

    # Elimination
    population = elimination(population, offspring, tsp)

    fitnesses = [fitness(tsp, ind) for ind in population]
    print(f'---------------------------------  ITERATION {i} ---------------------------------')
    print("Mean fitness:  ", statistics.mean(fitnesses))
    print("Best fitness:  ", min(fitnesses))

  # Print final Candidate solutions
  for ind in population:
    print("Candidate solution:  ", ind.order)
    print("Candidate mutation probability:  ", ind.α)


def initialize(tsp, λ):
  population = []
  for i in range(λ):
    population.append(Individual(tsp))
  return population

def mutate(individual):
  if np.random.rand() < individual.α:
    i = random.randrange(len(individual.order))
    j = random.randrange(len(individual.order))
    if i < j:
      individual.order[i:j] = individual.order[i:j][::-1]
    else:
      individual.order[j:i] = individual.order[j:i][::-1]

def recombination(parent1, parent2):
  child = parent1
  # Select crossover points randomly
  i = random.randrange(len(parent1.order))
  j = random.randrange(len(parent1.order))
  # Perform order crossover recombination to create the child's order
  if i < j:  
    child.order = order_crossover(parent1.order, parent2.order, i, j)
  else:  
    child.order = order_crossover(parent1.order, parent2.order, j, i)  
  # Set child parameters
  β = 2 * np.random.rand() - 0.5 # coefficient between (-0.5, 1.5)
  child.α = parent1.α + β * (parent2.α - parent1.α)
  return child

def order_crossover(parent1, parent2, i,j):
  # Create loop effect over array
  p1 = np.concatenate((parent1, parent1), axis=None)
  p2 = np.concatenate((parent2, parent2), axis=None)
  # Create empty child order
  child = np.full(10, None) 
  # Fill child with random selected sequence from parent1
  child[i:j] = p1[i:j]
  # Set pointers to the second crossover point in parent2 (jj) and child (ii)
  ii, jj = j, j
  # The recombination stops when all nodes in parent2 sequence have been added to the child
  while (jj < j + len(parent2)):
    if ii > 9:
      # when child's pointer reaches the end of the array go to position 0
      ii = 0
    if p2[jj] not in child: 
      # Missing nodes are added to the child
      child[ii] = p2[jj]
      ii += 1 # when added a new node, shift child's pointer to the left
    jj += 1
  return child

def selection(tsp, population):
  k = 5
  candidates = random.sample(population, k)
  fitnesses = [fitness(tsp, ind) for ind in candidates]
  selected = fitnesses.index(min(fitnesses))
  return candidates[selected]

def elimination(population, offspring, tsp):
  # λ+μ elimination
  combined = population + offspring
  ranked_individuals = sorted([(fitness(tsp, ind), ind) for ind in combined], key=lambda ind: ind[0])[0:len(population)]
  return [ind for fit, ind in ranked_individuals]

def matched_subseq(parent1, parent2):
  # Mejor ponerlos aleatoriamente para mas exploracion en vez de concat
  # CAREFUL [1,2,3,4] = [2,3,4,1]
  matchedSubseq = np.array([], dtype=int)
  window_size = 2
  window_pos = 0
  while window_pos < parent1.size:  
    # extract subsequence from parent
    subseq = parent1[window_pos:window_pos+window_size] #problem: last node is alone
    #print(subseq)
    # Search longest common subsequences
    matched = None
    num_matches = 0
    keepSearching = True
    while (keepSearching and window_pos+window_size <= parent1.size):
      # take the first individual again
      if search_sequence(parent2, subseq):
        # Save matching subsequence
        matched = subseq
        #print(f'Matched: {matched}')
        # If it is the last common subsequence we append it
        if window_pos+window_size is parent1.size: 
          matchedSubseq = np.concatenate((matchedSubseq, matched), axis=None)
          #print('appended subsequence:  ', matched)
        # Increase subsequence length
        num_matches += 1
        window_size += 1
        subseq = parent1[window_pos:window_pos+window_size]
        #print(f'{subseq} from {window_pos} to {window_pos+window_size}')
      else: 
        if matched is not None:
          matchedSubseq = np.concatenate((matchedSubseq, matched), axis=None)
        #print('appended subsequence:  ', matched)
        matched = None
        window_size = 2
        keepSearching = False
    window_pos += 1 + num_matches
  return matchedSubseq

def search_sequence(arr,seq):
    # Store sizes of input array and sequence
    Na, Nseq = arr.size, seq.size

    # Range of sequence
    r_seq = np.arange(Nseq)
    r_arr = np.arange(Na)
    # Create a 2D array of sliding indices across the entire length of input array.
    # Match up with the input sequence & get the matching starting indices.
    M = (arr[np.arange(Na-Nseq+1)[:,None] + r_seq] == seq).all(1)
    N = (seq[np.arange(Nseq-Na+1)[:,None] + r_arr] == arr).all(1)
    # Get the range of those indices as final output
    if M.any() >0 or N.any() > 0:
        return True
    else:
        return False         # No match found

In [26]:
# Order crossover
parent1 = np.array(range(0,10))
parent2 = np.array([9,3,7,8,2,6,5,1,4,0])
print(parent1)
print(parent2)
i = random.randrange(len(parent1))
j = random.randrange(len(parent1))
if i < j:  
  print(order_crossover(parent1, parent2, i, j))
else:  
  print(order_crossover(parent1, parent2, j, i))


[0 1 2 3 4 5 6 7 8 9]
[9 3 7 8 2 6 5 1 4 0]
[6 1 2 3 4 5 0 9 7 8]


In [ ]:
tsp = TSP(10)
population=initialize(tsp,15)
for i in population:
  print("Candidate solution:  ", i.order)
  print("Objective value: ", fitness(tsp, i))

parent1 = selection(tsp, population)
parent2 = selection(tsp, population)
print(f'\n Selected parent {parent1.order} y fitness: {fitness(tsp, parent1)}')
print(f'\n Selected parent {parent2.order} y fitness: {fitness(tsp, parent2)}')

offspring = recombination(parent1, parent2)
print(f'\n Offspring recombination {offspring.order} y fitness: {fitness(tsp, offspring)}')

mutate(offspring)
print(f'\n Offspring mutated {offspring.order} y fitness: {fitness(tsp, offspring)}')

# Mutation
for ind in population:
  mutate(ind)    

off = [offspring]
# Elimination
population = elimination(population, off, tsp)
for i in population:
  print("Candidate solution:  ", i.order)
  print("Objective value: ", fitness(tsp, i))


In [33]:
tsp = TSP(10)

#print("Vertices:  ", tsp.vertices)
#print("Distance:  \n", tsp.distanceMatrix)

optimize(tsp)

"""best = elimination(pop, pop, tsp)
for i in best:
  print("Candidate solution:  ", i.order)
  print("Objective value: ", fitness(tsp, i))

fitnesses = [fitness(tsp, ind) for ind in best]
print("\nMean fitness:  ", statistics.mean(fitnesses))
print("Best fitness:  ", min(fitnesses))
"""

Candidate solution:   [2 3 6 7 8 0 5 9 4 1]
Candidate solution:   [0 9 6 1 7 8 4 2 3 5]
Candidate solution:   [0 9 8 2 6 3 5 4 1 7]
Candidate solution:   [2 5 8 1 3 9 4 7 0 6]
Candidate solution:   [8 3 6 9 5 7 2 4 1 0]
Candidate solution:   [1 0 4 9 8 7 5 6 3 2]
Candidate solution:   [4 3 5 2 0 7 9 1 6 8]
Candidate solution:   [8 4 6 7 1 9 2 5 0 3]
Candidate solution:   [2 0 3 5 8 7 1 6 9 4]
Candidate solution:   [4 3 8 0 9 6 1 5 7 2]
Candidate solution:   [4 2 1 7 5 8 0 6 3 9]
Candidate solution:   [6 8 7 0 3 2 5 1 9 4]
Candidate solution:   [4 7 5 0 6 2 3 1 9 8]
Candidate solution:   [6 5 9 3 2 4 8 7 0 1]
Candidate solution:   [8 2 3 7 0 9 5 4 1 6]
Candidate solution:   [9 7 5 1 6 0 4 2 3 8]
Candidate solution:   [8 9 1 4 2 0 3 5 7 6]
Candidate solution:   [4 6 2 1 0 9 3 7 5 8]
Candidate solution:   [6 5 3 2 0 7 4 8 1 9]
Candidate solution:   [7 1 8 2 0 6 4 9 3 5]
Candidate solution:   [4 6 3 0 9 8 5 2 1 7]
Candidate solution:   [4 3 0 7 9 5 2 1 8 6]
Candidate solution:   [9 8 4 5 7

'best = elimination(pop, pop, tsp)\nfor i in best:\n  print("Candidate solution:  ", i.order)\n  print("Objective value: ", fitness(tsp, i))\n\nfitnesses = [fitness(tsp, ind) for ind in best]\nprint("\nMean fitness:  ", statistics.mean(fitnesses))\nprint("Best fitness:  ", min(fitnesses))\n'

In [ ]:
#a = np.random.randint(1, 10, size=(4, 4))
# np.fill_diagonal(a, 0)
a = np.array([5,1,2,3,4,6])
parent1 = np.array([1,2,3,5,4,7,6])
parent2 = np.array([1,2,3,5,4,6,7])
parent3 = np.array([5,1,2,0,3,4])
parent4 = np.array([1,2,5,3,4,0])
parent5 = np.array([5,1,2,6,3,4])
parent6 = np.array([5,2,1,3,6,4])
print(f'Parent1: {parent1} Parent2: {parent2}')
print(f'Parent3: {parent3} Parent4: {parent4}')
print(f'Parent5: {parent5} Parent6: {parent6}')

def matched_subseq(parent1, parent2):
  # Mejor ponerlos aleatoriamente para mas exploracion en vez de concat
  # CAREFUL [1,2,3,4] = [2,3,4,1]
  matchedSubseq = np.array([], dtype=int)
  window_size = 2
  window_pos = 0
  while window_pos < parent1.size:  
    # extract subsequence from parent
    subseq = parent1[window_pos:window_pos+window_size] #problem: last node is alone
    #print(subseq)
    # Search longest common subsequences
    matched = None
    num_matches = 0
    keepSearching = True
    while (keepSearching and window_pos+window_size <= parent1.size):
      # take the first individual again
      if search_sequence(parent2, subseq):
        # Save matching subsequence
        matched = subseq
        #print(f'Matched: {matched}')
        # If it is the last common subsequence we append it
        if window_pos+window_size is parent1.size: 
          matchedSubseq = np.concatenate((matchedSubseq, matched), axis=None)
          #print('appended subsequence:  ', matched)
        # Increase subsequence length
        num_matches += 1
        window_size += 1
        subseq = parent1[window_pos:window_pos+window_size]
        #print(f'{subseq} from {window_pos} to {window_pos+window_size}')
      else: 
        if matched is not None:
          matchedSubseq = np.concatenate((matchedSubseq, matched), axis=None)
        #print('appended subsequence:  ', matched)
        matched = None
        window_size = 2
        keepSearching = False
    window_pos += 1 + num_matches
  return matchedSubseq
print(f'Appended sequences: {matched_subseq(parent1, parent2)}')
print(f'Appended sequences: {matched_subseq(parent3, parent4)}')
print(f'Appended sequences: {matched_subseq(parent5, parent6)}')

sharedCycles = matched_subseq(parent3, parent4)
order = sharedCycles
node = 0
while (node < parent3.size):
  if not search_sequence(np.array([node]), sharedCycles):
    if np.random.rand() < 0.5:
      order = np.concatenate((order, np.array(node)), axis=None)
    else:
      order = np.concatenate((np.array(node), order), axis=None)
  node += 1
order

Parent1: [1 2 3 5 4 7 6] Parent2: [1 2 3 5 4 6 7]
Parent3: [5 1 2 0 3 4] Parent4: [1 2 5 3 4 0]
Parent5: [5 1 2 6 3 4] Parent6: [5 2 1 3 6 4]
Appended sequences: [1 2 3 5 4]
Appended sequences: [1 2 3 4]
Appended sequences: []


array([1, 2, 3, 4, 0, 5])

In [ ]:
a = np.array([5,1,2,3,4,6])
parent1 = np.array([1,2,3,5,4,7,6])
parent2 = np.array([1,2,3,5,4,6,7])
parent3 = np.array([5,1,2,6,3,4])
parent4 = np.array([1,2,5,3,4,6])
parent5 = np.array([5,1,2,6,3,4])
parent6 = np.array([5,1,2,3,6,4])
print(f'Parent1: {parent1} Parent2: {parent2}')
print(f'Parent3: {parent3} Parent4: {parent4}')
print(f'Parent5: {parent5} Parent6: {parent6}')

def matched_subseq(parent1, parent2):
  matchedSubseq = []
  window_size = 2
  window_pos = 0
  if parent1[0] == parent2[0] and parent1[-1] == parent2[-1]:
    parent1 = np.concatenate((parent1, parent1), axis=None)
    parent2 = np.concatenate((parent2, parent2), axis=None)
  while window_pos < parent1.size:  
    # extract subsequence from parent
    subseq = parent1[window_pos:window_pos+window_size] #problem: last node is alone
    #print(subseq)
    # Search longest common subsequences
    matched = None
    num_matches = 0
    keepSearching = True
    while (keepSearching and window_pos+window_size <= parent1.size):
      # take the first individual again
      if search_sequence(parent2, subseq):
        # Save matching subsequence
        matched = subseq
        #print(f'Matched: {matched}')
        # If it is the last common subsequence we append it
        if window_pos+window_size is parent1.size: 
          matchedSubseq.append(matched)
          #print('appended subsequence:  ', matched)
        # Increase subsequence length
        num_matches += 1
        window_size += 1
        subseq = parent1[window_pos:window_pos+window_size]
        #print(f'{subseq} from {window_pos} to {window_pos+window_size}')
      else: 
        if matched is not None:
          matchedSubseq.append(matched)
        #print('appended subsequence:  ', matched)
        matched = None
        window_size = 2
        keepSearching = False
    window_pos += 1 + num_matches
  return matchedSubseq
print(f'Appended sequences: {matched_subseq(parent1, parent2)}')
print(f'Appended sequences: {matched_subseq(parent3, parent4)}')
print(f'Appended sequences: {matched_subseq(parent5, parent6)}')
#parent5[(4+2)%parent5.size]
problemita = matched_subseq(parent5, parent6)
search_sequence(problemita[0], problemita[1]) 
"""for i in range(len(sharedCycles)):
  if i+1 < len(sharedCycles):
    order = np.concatenate((sharedCycles[i], sharedCycles[i+1]), axis=None)
  i += 1
print(order)"""

Parent1: [1 2 3 5 4 7 6] Parent2: [1 2 3 5 4 6 7]
Parent3: [5 1 2 6 3 4] Parent4: [1 2 5 3 4 6]
Parent5: [5 1 2 6 3 4] Parent6: [5 1 2 3 6 4]
Appended sequences: [array([1, 2, 3, 5, 4])]
Appended sequences: [array([1, 2]), array([3, 4])]
Appended sequences: [array([5, 1, 2]), array([4, 5, 1, 2])]


True